 # Agente LangChain - Notícias do seu time do coração



 Este notebook implementa um agente que coleta notícias do seu time e as processa usando LLM no OCI.



 ## Funcionalidades:

 - Coleta notícias do GE Globo

 - Processa com LLM via OCI

 - Busca inteligente com LangChain

### Modifique para seu time do coração

In [6]:
time = "Flamengo"

 ## Coletor de notícias do seu time

In [2]:
!pip install tavily-python
!pip install langchain-community langchain-core oci



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: C:\Users\Amanda Machado\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: C:\Users\Amanda Machado\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [21]:
import requests
from langchain_core.tools import tool
from typing import Dict, List
import re
from tavily import TavilyClient

@tool
def get_time_comprehensive_news(pergunta:str=f"Qual proximo jogo do {time}?") -> str:
    """Retorna notícias do time de futebol de múltiplas fontes do GE Globo"""
    
    client = TavilyClient("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") ## Pode usar esse mesmo, tem limite de 100 requests para o workshop
    response = client.search(
    query=pergunta)
    return response



## Adicionando tool de data atual

In [22]:
@tool
def get_today_date() -> str:
    """Retorna a data de hoje no formato DD/MM/AAAA"""
    from datetime import datetime
    return datetime.now().strftime("%d/%m/%Y")

 ## Configuração de autenticação de OCI SDK
    - https://github.com/oracle/oci-python-sdk/blob/master/examples/configuration_example.py

 ## Criação do Agente LangChain

In [23]:

from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_community.chat_models import ChatOCIGenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate

# 2. Configuração da LLM OCI
llm = ChatOCIGenAI(
  model_id="ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyajqi26fkxly6qje5ysvezzrypapl7ujdnqfjq6hzo2loq",
  service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
  compartment_id="ocid1.compartment.oc1..aaaaaaaa3x7n7wwfnghe4imvt3niwo76wgqv6ecn2iadiwoph73jjowbhbna",
  provider="meta",
  model_kwargs={
    "temperature": 0.3, 
    "max_tokens": 800,   
    "top_p": 0.8,  
    "frequency_penalty": 0,
    "presence_penalty": 0,
  },
  auth_type="API_KEY",
  auth_profile="CHICAGOV2"
)
tools = [
    get_time_comprehensive_news,
    get_today_date
]

react_prompt_template = """You are a helpful assistant specialized in Brazilian football teams news and information. 
Answer the following questions as best you can using the available tools.

IMPORTANT: After using any tool, you MUST analyze and summarize the results in your own words. 
Do not just copy the raw output - provide a meaningful summary and insights. Do not use old news.

You have access to the following tools:

{tools}

Use the following format EXACTLY:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action (use empty string "" if no input needed)
Observation: the result of the action
Thought: Now I need to analyze this information and provide a helpful summary
Final Answer: A comprehensive answer based on the tool results, with analysis and insights in Brazilian Portuguese

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(react_prompt_template)

agent = create_react_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    max_iterations=3  # Limita o número de iterações para evitar loops infinitos
)
pergunta = f"Qual próximo jogo do {time}?"

resposta = agent_executor.invoke({"input": pergunta})




> Entering new AgentExecutor chain...
Para responder a essa pergunta, preciso saber qual é o próximo jogo do Flamengo. 
Action: get_time_comprehensive_news
Action Input: Qual proximo jogo do Flamengo?{'query': 'Qual proximo jogo do Flamengo?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.placardefutebol.com.br/time/flamengo/proximos-jogos', 'title': 'Próximos jogos do Flamengo - calendário, datas e horários', 'content': 'Próximos jogos do Flamengo ; Flamengo. Estudiantes. amanhã 21:30 ; Flamengo. Vasco. DOM, 21/09 17:30 ; Estudiantes. Flamengo. QUI, 25/09 21:30 ; Corinthians.', 'score': 0.8891543, 'raw_content': None}, {'url': 'https://www.flamengo.com.br/jogos', 'title': 'Próximos jogos - Clube de Regatas do Flamengo', 'content': 'Próximos jogos Image 1: KwaiFacebookImage 2: XInstagramInstagramYoutubeYoutubeTikTokTikTok Seja Sócio-torcedor Aproveite o Fla Prêmios Image 5: KwaiFacebookFacebookImage 6: XInstagramInstagramYoutubeYoutubeTik

### Apresentando resposta do agente de forma mais amigável

In [14]:
from IPython.display import Markdown
display(Markdown(resposta['output']))

O próximo jogo do Flamengo é contra o Grêmio, marcado para o dia 31 de agosto de 2025, às 16:00, no Estádio Jornalista Mário Filho, pelo Campeonato Brasileiro. Além disso, o time tem outros jogos programados para setembro e outubro, incluindo partidas contra Vasco da Gama, Cruzeiro, Palmeiras, Juventude, Bahia e Botafogo. É sempre uma boa ideia verificar as fontes oficiais ou sites de notícias esportivas para obter as informações mais atualizadas.

In [15]:
analysis_prompt = f"""
Com base nas seguintes informações sobre o {time}, forneça um resumo claro e útil:

DADOS COLETADOS:
{resposta}

Por favor, analise essas informações e forneça:
1. Resposta direta à pergunta: "{pergunta}"

Responda de forma conversacional e útil, não apenas liste os dados.
"""

analysis_response = llm.invoke([HumanMessage(content=analysis_prompt)])
display(Markdown(analysis_response.content))

Se você está procurando saber sobre o próximo jogo do Flamengo, posso ajudar! De acordo com as informações disponíveis, o próximo jogo do Flamengo está marcado para o dia 31 de agosto de 2025, às 16:00, e será contra o Grêmio. Esse jogo acontecerá no Estádio Jornalista Mário Filho e faz parte do Campeonato Brasileiro.

Além disso, é importante saber que o time tem uma agenda bem movimentada nos meses seguintes, com jogos contra outros times importantes como Vasco da Gama, Cruzeiro, Palmeiras, Juventude, Bahia e Botafogo, tanto em setembro quanto em outubro. No entanto, para ter as informações mais atualizadas e confirmadas, sempre é uma boa ideia verificar as fontes oficiais do clube ou sites especializados em notícias esportivas. Assim, você pode ter certeza de que está acompanhando a programação do Flamengo de forma precisa e atualizada.

### Perguntando para o agente

In [24]:
pergunta = f"De quanto foi o ultimo jogo?"

resposta = agent_executor.invoke({"input": pergunta})
display(Markdown(resposta['output']))



> Entering new AgentExecutor chain...
Question: De quanto foi o ultimo jogo?
Thought: Para responder a essa pergunta, preciso saber a data do último jogo do time em questão, no entanto, a pergunta não especifica o time. Considerando que o usuário está perguntando sobre o último jogo, é provável que esteja se referindo a um time específico, mas como não foi mencionado, vou considerar um time popular como o Flamengo para ter uma base para a resposta. Primeiro, preciso saber a data de hoje para ter uma referência para procurar notícias recentes.

Action: get_today_date
Action Input: 18/09/2025Agora que sei a data de hoje, posso usar essa informação como referência para procurar notícias recentes sobre o último jogo do Flamengo. Considerando que a pergunta não especifica o time, vou focar no Flamengo como exemplo, mas é importante notar que a resposta pode variar dependendo do time em questão.

Action: get_time_comprehensive_news
Action Input: Qual foi o último jogo do Flamengo?{'query':

O último jogo do Flamengo pode ter sido uma derrota por 1 a 3 para o Internacional na Copa Libertadores, ou uma vitória por 2 a 1 sobre o Mirassol no Campeonato Brasileiro, dependendo da data e da competição considerada. É importante notar que a resposta pode variar dependendo do contexto e da fonte de informação.